<a href="https://colab.research.google.com/github/VedvratGautam/Learning-Machines/blob/Portfolio/football_match_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Football Matches

In this project I will try to predict results of football matches in the English Premier League.

The data is from the 21-22 and 22-23(on going at the time) season.

Predicting sports is extremely difficult but this might give me some insights as to how bookmakers can give odds for matches and make a consistent income.

I simply chose features that had the highest correlation with the target.
<br>
<br>
NOTE : <br>

- Many features are chosen as I am very familiar with Football and have a good level of domain knowledge.

- Many of these features are statistics that are calculated after the match or during the match. Therefore this model is not a great indicator of predicting games beforehand.  

In [442]:
#Libraries

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [443]:
matches = pd.read_csv('/content/matches.csv' , index_col = 0)

In [444]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [445]:
matches.shape

(1389, 27)

## Investigating Missing Data

In [446]:
38 * 20 * 2

1520

There are a total of 20 teams each playing 38 games a season. We have data for 2 seasons, therefore we should have 1520 rows, but we have 1389 rows meaning some data is missing. Lets investigate.

In [447]:
matches['team'].value_counts()

,count
team,
Manchester United,72
West Ham United,72
Newcastle United,72
Brighton and Hove Albion,72
Southampton,72
Tottenham Hotspur,71
Manchester City,71
Leeds United,71
Wolverhampton Wanderers,71


3 teams are relegated each season and 3 teams are promoted. Given that these 3 were different for the consecutive seasons, we can assume a maximum of upto 6 teams having less than 70ish matches played.

In [448]:
#Liverpool was not relegated

matches[matches['team'] == 'Liverpool']

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2020-09-12,17:30,Premier League,Matchweek 1,Sat,Home,W,4.0,3.0,Leeds United,...,Match Report,NaN,20.0,4.0,17.0,0.0,2.0,2.0,2021,Liverpool
2,2020-09-20,16:30,Premier League,Matchweek 2,Sun,Away,W,2.0,0.0,Chelsea,...,Match Report,NaN,17.0,5.0,17.7,1.0,0.0,0.0,2021,Liverpool
4,2020-09-28,20:00,Premier League,Matchweek 3,Mon,Home,W,3.0,1.0,Arsenal,...,Match Report,NaN,21.0,9.0,16.8,0.0,0.0,0.0,2021,Liverpool
6,2020-10-04,19:15,Premier League,Matchweek 4,Sun,Away,L,2.0,7.0,Aston Villa,...,Match Report,NaN,14.0,8.0,15.8,1.0,0.0,0.0,2021,Liverpool
7,2020-10-17,12:30,Premier League,Matchweek 5,Sat,Away,D,2.0,2.0,Everton,...,Match Report,NaN,22.0,8.0,15.0,1.0,0.0,0.0,2021,Liverpool
9,2020-10-24,20:00,Premier League,Matchweek 6,Sat,Home,W,2.0,1.0,Sheffield Utd,...,Match Report,NaN,17.0,5.0,18.2,1.0,0.0,0.0,2021,Liverpool
11,2020-10-31,17:30,Premier League,Matchweek 7,Sat,Home,W,2.0,1.0,West Ham,...,Match Report,NaN,8.0,2.0,18.6,1.0,1.0,1.0,2021,Liverpool
13,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Away,D,1.0,1.0,Manchester City,...,Match Report,NaN,9.0,2.0,21.5,0.0,1.0,1.0,2021,Liverpool
14,2020-11-22,19:15,Premier League,Matchweek 9,Sun,Home,W,3.0,0.0,Leicester City,...,Match Report,NaN,24.0,12.0,11.9,0.0,0.0,0.0,2021,Liverpool
16,2020-11-28,12:30,Premier League,Matchweek 10,Sat,Away,D,1.0,1.0,Brighton,...,Match Report,NaN,6.0,2.0,20.9,0.0,0.0,0.0,2021,Liverpool


We only have data for one season for Liverpool

In [449]:
matches['round'].value_counts()

,count
round,
Matchweek 1,39
Matchweek 2,39
Matchweek 3,39
Matchweek 4,39
Matchweek 5,39
Matchweek 6,39
Matchweek 7,39
Matchweek 8,39
Matchweek 9,39


Since data was scraped mid-season, there is some missing data.

## Cleaning the Data

In [450]:
matches.dtypes

,0
date,object
time,object
comp,object
round,object
day,object
venue,object
result,object
gf,float64
ga,float64
opponent,object


In [451]:
#Changing data column datatype to datetime

matches['date'] = pd.to_datetime(matches['date'])
matches.dtypes

,0
date,datetime64[ns]
time,object
comp,object
round,object
day,object
venue,object
result,object
gf,float64
ga,float64
opponent,object


In [452]:
#Dropping duplicate values

matches.drop_duplicates()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,...,Match Report,NaN,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,...,Match Report,NaN,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,...,Match Report,NaN,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United


In [453]:
#Checking for na values

matches.isna().sum()

,0
date,0
time,0
comp,0
round,0
day,0
venue,0
result,0
gf,0
ga,0
opponent,0


Notes column is useless and can be dropped where as attendance may be helpful factor later on as home crowd attendance may have some influence on games.

In [454]:
#Dropping notes column entirely

matches.drop(columns = 'notes' , inplace = True)

In [455]:
matches.shape

(1389, 26)

In [456]:
matches['target'] = (matches['result'] == 'W').astype('int')

## EDA to find out which features may be useful in creating our initial model

Here I will try to include the most obvious features from the dataset first.

Visualize trends and answer questions.

In [457]:
matches.nunique()

,0
date,244
time,18
comp,1
round,38
day,7
venue,2
result,3
gf,9
ga,9
opponent,23


In [458]:
matches['target']

,target
1,0
2,1
3,1
4,1
6,0
...,...
38,0
39,0
40,1
41,0


Figuring out which features, if a higher value means the team is more likely to win.

In [459]:
matches.groupby('target').mean(numeric_only= True)

,gf,ga,xg,xga,poss,attendance,sh,sot,dist,fk,pk,pkatt,season
target,,,,,,,,,,,,,
0,0.687138,1.880649,1.040904,1.567903,48.607184,36172.587156,11.214368,3.282735,17.216241,0.448436,0.063731,0.093859,2021.463499
1,2.399240,0.560837,1.736122,0.961977,51.500000,35949.793774,13.693916,5.285171,16.676046,0.467681,0.211027,0.235741,2021.435361


Features that have higher values leading to a team winning -> gf, xg, sh, sot, pk

In [460]:
numeric_matches = matches.select_dtypes('number')
numeric_matches.corr()['target'].sort_values(ascending= False)

,target
target,1.000000
gf,0.651977
xg,0.439669
sot,0.404204
sh,0.228353
pk,0.202941
pkatt,0.177529
poss,0.113184
fk,0.014034
attendance,-0.006051


## Creating Predictors

Venue has a big influence on deciding games. It has only two values Home and Away. I will use it as one of the predictors.

In [461]:
matches['venue_code'] = matches['venue'].astype('category').cat.codes

In [462]:
# 0 --> Away Venue
# 1 --> Home Venue

matches['venue_code']

,venue_code
1,0
2,1
3,1
4,0
6,1
...,...
38,0
39,1
40,0
41,0


Creating codes for opponent teams as well. This is just done for my convinience and will not provide it to the ML model as it may consider the weight of team with code 20 to be 2x the weight of team with code 10.

In [463]:
matches['opp_code'] = matches['opponent'].astype('category').cat.codes

In [464]:
# Each team is assigned a unique code

matches['opp_code']

,opp_code
1,18
2,15
3,0
4,10
6,17
...,...
38,18
39,6
40,7
41,14


Hour can be an interseting feature to consider. Some teams might win more games during a certain hours as opposed to other hours.

In [465]:
matches['hour'] = matches['time'].replace(':.+' , '' , regex = True).astype('int')

In [466]:
matches['hour']

,hour
1,16
2,15
3,12
4,15
6,15
...,...
38,19
39,15
40,19
41,18


Some teams might play better on mid-week fixtures than weekend fixtures. This can be due to a variety of causes.

In [467]:
# Creating a day code column

matches['day_code'] = matches['date'].dt.dayofweek

In [468]:
matches['day_code']

,day_code
1,6
2,5
3,5
4,5
6,5
...,...
38,6
39,5
40,6
41,2


## Creating target column

My current objective is to predicit the winner of the match. So I do not care about losses and draws at the moment.

In [469]:
matches['target']

,target
1,0
2,1
3,1
4,1
6,0
...,...
38,0
39,0
40,1
41,0


## Creating Initial ML model

Using Random Forest Classifier

In [470]:
#Initialising model

rf = RandomForestClassifier(n_estimators=50, min_samples_split = 10, random_state = 1)

In [471]:
# Splitting training and testing data

train = matches[matches['date'] < '2022-01-01'] #Time series data
train.shape

(1107, 31)

In [472]:
test = matches[matches['date'] > '2022-01-01' ]
test.shape

(276, 31)

In [473]:
predictors = ['venue_code' , 'opp_code' , 'hour' , 'day_code' , 'xg' , 'gf' ,'sot']

In [474]:
# Fitting the model

rf.fit(train[predictors] , train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [475]:
#Predictions

preds = rf.predict(test[predictors])

## Checking Accuracy and Precision

In [476]:
acc = accuracy_score(test['target'] , preds)

In [477]:
acc


0.822463768115942

With our current model we have a 82% accuracy score. Not bad but not great either.

I will now dig deeper to see in which situations our model was most accurate.

In [478]:
# Creating a combined Dataframe with predictions and actual values

combined = pd.DataFrame(dict(actual = test['target'] , prediction = preds))

In [479]:
combined

,actual,prediction
31,1,0
32,0,0
34,1,1
35,1,1
37,0,1
...,...,...
33,0,0
34,0,0
35,1,1
36,0,1


In [480]:
# Comparing predictions to actual results

pd.crosstab(index = combined['actual'], columns = combined['prediction'])

prediction,0,1
actual,,
0,152,20
1,29,75


Our model is makes correct predictions when it predicts a loss or a draw as opposed to a win.

Since we initally targetted predicting wins, lets see our precision score.

In [481]:
precision_score(test['target'] , preds)

0.7894736842105263

78% precision. This means we have chosen good features.

## Creating more predictors

Creating more predictors to imporve accuracy and precision.

In [482]:
#This allows me to look at data team wise

grouped_matches = matches.groupby('team')

In [483]:
group = grouped_matches.get_group('Arsenal')
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,target,venue_code,opp_code,hour,day_code
0,2021-08-13,20:00,Premier League,Matchweek 1,Fri,Away,L,0.0,2.0,Brentford,...,1.0,0.0,0.0,2022,Arsenal,0,0,2,20,4
1,2021-08-22,16:30,Premier League,Matchweek 2,Sun,Home,L,0.0,2.0,Chelsea,...,0.0,0.0,0.0,2022,Arsenal,0,1,5,16,6
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Away,L,0.0,5.0,Manchester City,...,0.0,0.0,0.0,2022,Arsenal,0,0,12,12,5
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,1.0,0.0,0.0,2022,Arsenal,1,1,15,15,5
5,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,1.0,0.0,0.0,2022,Arsenal,1,0,4,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2021-05-02,14:00,Premier League,Matchweek 34,Sun,Away,W,2.0,0.0,Newcastle Utd,...,1.0,0.0,0.0,2021,Arsenal,1,0,14,14,6
54,2021-05-09,19:00,Premier League,Matchweek 35,Sun,Home,W,3.0,1.0,West Brom,...,1.0,0.0,0.0,2021,Arsenal,1,1,20,19,6
55,2021-05-12,20:15,Premier League,Matchweek 36,Wed,Away,W,1.0,0.0,Chelsea,...,0.0,0.0,0.0,2021,Arsenal,1,0,5,20,2
56,2021-05-19,19:00,Premier League,Matchweek 37,Wed,Away,W,3.0,1.0,Crystal Palace,...,0.0,0.0,0.0,2021,Arsenal,1,0,6,19,2


In [484]:
#Creating a function that can calculate averages stats for team's certain features uptil a certain date

def rolling_averages(group , cols, new_cols):
  group = group.sort_values('date')
  rolling_stats = group[cols].rolling(3, closed = 'left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset = new_cols)
  return group

I want to include more relevant features.

In [485]:
cols = ['gf' , 'ga' , 'sh' , 'sot' , 'dist' , 'fk' , 'pk' , 'pkatt' , 'xg']
new_cols = [f'{c}_rolling' for c in cols]

Features chosen:

gf -> goals for or goals scored by team <br>
ga -> goals against or goals scored against the team <br>
sh -> shots taken by the team <br>
sot -> shots that were on target <br>
dist -> distance of the shots from the goal <br>
fk -> freekicks awarded <br>
pk -> penalty kicks <br>
pkatt -> penalty kick attempts <br>

In [486]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling',
 'xg_rolling']

In [487]:
rolling_averages(group , cols , new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,xg_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000,1.466667
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000,1.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000,0.833333
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000,0.733333
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333,0.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2022-04-04,20:00,Premier League,Matchweek 31,Mon,Away,L,0.0,3.0,Crystal Palace,...,0,1.000000,0.666667,13.000000,3.666667,18.400000,0.000000,0.333333,0.333333,1.366667
36,2022-04-09,15:00,Premier League,Matchweek 32,Sat,Home,L,1.0,2.0,Brighton,...,5,0.333333,1.666667,10.333333,2.666667,19.300000,0.000000,0.000000,0.000000,0.866667
37,2022-04-16,15:00,Premier League,Matchweek 33,Sat,Away,L,0.0,1.0,Southampton,...,5,0.666667,1.666667,14.000000,3.333333,17.566667,0.666667,0.000000,0.000000,1.300000
38,2022-04-20,19:45,Premier League,Matchweek 25,Wed,Away,W,4.0,2.0,Chelsea,...,2,0.333333,2.000000,18.333333,4.333333,17.400000,1.000000,0.000000,0.000000,1.433333


In [488]:
# Applying rolling averages to all matches

matches_rolling = matches.groupby('team').apply(lambda team: rolling_averages(team, cols, new_cols))

/tmp/ipython-input-1416850646.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby('team').apply(lambda team: rolling_averages(team, cols, new_cols))


In [489]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 32 2022-03-13  14:00  Premier League  Matchweek 29   
                        33 2022-03-18  20:00  Premier League  Matchweek 30   
                        34 2022-04-02  15:00  Premier League  Matchweek 31   
                        35 2022-04-08  20:00  Premier League  Matchweek 32   
                        36 2022-04-24  14:00  Premier League  Matchweek 34   

                            day venue result   gf   ga         opponent  ...  \
team                                                                     ...   
Arsenal                 6   Sun  Home      W  2.0  1.0    Sheffield Utd  ...   
                        7   Sat  Away      L  0.0  1.0  Manchester City  ...   
                        9   Sun  Home      L  0.0  1.0   Leicester City  ...   
                        11  Sun  Away      W  1.0  0.0   Manchester Utd  ...   
                        13  Sun  Home      L  0.0  3.0      Aston Villa  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 32  Sun  Away      W  1.0  0.0          Everton  ...   
                        33  Fri  Home      L  2.0  3.0     Leeds United  ...   
                        34  Sat  Home      W  2.0  1.0      Aston Villa  ...   
                        35  Fri  Away      L  0.0  1.0    Newcastle Utd  ...   
                        36  Sun  Away      L  0.0  1.0          Burnley  ...   

                            day_code  gf_rolling  ga_rolling  sh_rolling  \
team                                                                       
Arsenal                 6          6    2.000000    1.333333    7.666667   
                        7          5    1.666667    1.666667    5.333333   
                        9          6    1.000000    1.666667    7.000000   
                        11         6    0.666667    1.000000    9.666667   
                        13         6    0.333333    0.666667    9.666667   
...                              ...         ...         ...         ...   
Wolverhampton Wanderers 32         6    1.333333    1.000000   12.333333   
                        33         4    1.666667    0.666667   12.333333   
                        34         5    2.333333    1.000000   13.000000   
                        35         4    1.666667    1.333333   13.000000   
                        36         6    1.333333    1.666667   10.000000   

                           sot_rolling dist_rolling fk_rolling pk_rolling  \
team                                                                        
Arsenal                 6     3.666667    14.733333   0.666667   0.000000   
                        7     3.666667    15.766667   0.000000   0.000000   
                        9     3.666667    16.733333   0.666667   0.000000   
                        11    4.000000    16.033333   1.000000   0.000000   
                        13    2.666667    18.033333   1.000000   0.333333   
...                                ...          ...        ...        ...   
Wolverhampton Wanderers 32    3.666667    19.300000   0.000000   0.000000   
                        33    4.333333    19.600000   0.000000   0.000000   
                        34    5.333333    19.833333   0.000000   0.000000   
                        35    5.000000    18.533333   0.000000   0.000000   
                        36    4.666667    17.633333   0.000000  

In [490]:
# Dropping extra index level

matches_rolling = matches_rolling.droplevel('team')

In [491]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,xg_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000,1.466667
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000,1.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000,0.833333
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000,0.733333
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333,0.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,Everton,...,6,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000,0.733333
33,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,Leeds United,...,4,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000,0.733333
34,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,Aston Villa,...,5,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000,0.833333
35,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,Newcastle Utd,...,4,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000,0.933333


In [492]:
matches_rolling.shape

(1317, 40)

In [493]:
#Removing duplicate indices

matches_rolling.index = range(matches_rolling.shape[0])

In [494]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,xg_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000,1.466667
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000,1.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000,0.833333
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000,0.733333
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333,0.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,Everton,...,6,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000,0.733333
1313,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,Leeds United,...,4,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000,0.733333
1314,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,Aston Villa,...,5,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000,0.833333
1315,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,Newcastle Utd,...,4,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000,0.933333


## Retraining ML model

Now we have new predictors with our rolling average features for the teams, lets see if our prediction accuracy and precision score imporve.

In [495]:
#Function to make predictions (did not want to be repetitive)

def make_predictions(data, predictors):

  #Splitting into training and testing data
  train = data[data['date'] < '2022-01-01']
  test = data[data['date'] > '2022-01-01']

  #Fitting into ML Model and getting predictions
  rf.fit(train[predictors] , train['target'])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual = test['target'] , predicted = preds), index = test.index)

  #Checking accuracy and precision
  accuracy = accuracy_score(test['target'] , preds)
  precision = precision_score(test['target'] , preds)

  return combined, accuracy, precision



In [496]:
combined , accuracy, precision = make_predictions(matches_rolling, predictors+new_cols)

In [497]:
accuracy

0.8405797101449275

In [498]:
precision

0.8061224489795918

Accuracy and precision both have risen by ~2%. This is not a very significant increase but calculation of rolling averages has certainly made an imporvement.

In [499]:
combined.value_counts()

actual  predicted
0       0            153
1       1             79
        0             25
0       1             19
Name: count, dtype: int64

In [500]:
combined = combined.merge(matches_rolling[['date' , 'team' , 'opponent' , 'result']] , left_index=True, right_index=True)

In [501]:
combined

,actual,predicted,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D
56,1,0,2022-02-10,Arsenal,Wolves,W
57,1,1,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,0,1,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,1,1,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


## Combining Home and Away Predictions

Currently our model was provided with data with matches including X vs Y and Y vs X within the same matchweek.
Therefore it is possible that it has predicted the same match with different results.

I will combine the predictions.

I have also noticed that some teams have different names in the team and opponent columns. Therefore they must be normalised.

In [502]:
class MissingDict(dict):
  __missing__ = lambda self , key: key   #Avoiding keyerrors and not returning empty values

map_values = {
    'Brighton and Hove Albion' : 'Brighton',
    'Manchester United' : 'Manchester Utd',
    'Newcastle United' : 'Newcastle Utd',
    'Tottenham Hotspur' : 'Tottenham',
    'West Ham United' : 'West Ham',
    'Wolverhampton Wanderers' : 'Wolves'
}

mapping = MissingDict(**map_values)

In [503]:
mapping['Arsenal']
mapping['Manchester United']

'Manchester Utd'

In [504]:
combined['new_team'] = combined['team'].map(mapping)

In [505]:
combined

,actual,predicted,date,team,opponent,result,new_team
55,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal
56,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal
57,1,1,2022-02-19,Arsenal,Brentford,W,Arsenal
58,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal
59,1,1,2022-03-06,Arsenal,Watford,W,Arsenal
...,...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves
1313,0,1,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves
1314,1,1,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves


Now we can use 'new_team' as the key to merge the dataframe with itself.

I am doing this so that I can see how the model performs when predicting the same matches but reversed fixture.


In [506]:
merged = combined.merge(combined, left_on = ['date' , 'new_team'] , right_on = ['date' , 'opponent'])

In [507]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,1,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,1,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,0,0,Everton,Wolves,L,Everton
258,0,1,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,1,1,Leeds United,Wolves,W,Leeds United
259,1,1,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,0,0,Aston Villa,Wolves,L,Aston Villa
260,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd


In [508]:
#Selecting rows where both side prediction was correct

merged[(merged['predicted_x'] == 1) & (merged['predicted_y'] == 0) ]['actual_x'].value_counts()

,count
actual_x,
1,69
0,1


In [510]:
69 / 70

0.9857142857142858

This shows that our model makes correct predictions for both sides of the coin. This means if for fixture X vs Y our model predicts that X will win, it is very likely to predict that Y will lose the same reverse fixture Y vs X.

# Future Scope of this Project

I would like to include more recent premier league data and a larger dataset. <br>
<br>
I also want to incorporate other relavent features  such as big chances created and find a way to include player ratings into the model as well. This will definately imporve the accuracy and precision score as this is a game heavily incfluenced by chances and individual players.